In [1]:
import pandas as pd
import json

- import list of all current (circa 2020), english, and public subreddits

In [4]:
subreddit_df = pd.read_csv("../Data/master_subreddit_list.csv")
subreddit_df.head()

,title,description,subscribers,created_utc,name
0,Not Safe for Work,#### AVOID REPOSTS!!! Check [KarmaDecay.com](...,1880887,1.137700e+09,nsfw
1,features,"For US news there is /r/news, for world news t...",3868,1.140332e+09,features
2,Request,\n\n***\n*Original sidebar content:*\n\n\n&gt;...,1588,1.140333e+09,request
3,"Citius, Altius, Fortius",&gt; [PyeongChang 2018 Website](https://www.py...,417898,1.140336e+09,olympics
4,/r/hanguk,[Click anywhere to continue to /r/hanguk](/r/h...,233,1.140417e+09,ko


- import results from query of GARD Diseases in neo4j database

In [5]:
neo4j_df = pd.read_csv('../Data/neo4j_rare_disease_list.csv')
neo4j_df.head()

,d.gard_id,d.is_rare,d.name,d.synonyms
0,GARD:0000001,True,GRACILE syndrome,"[FLNMS,Finnish lactic acidosis with hepatic he..."
1,GARD:0000003,True,Ablepharon macrostomia syndrome,"[AMS,Congenital ablepharon, absent eyelashes/e..."
2,GARD:0000004,True,Acanthocheilonemiasis,"[Dipetalonemiasis,Dipetalonema infections,Acan..."
3,GARD:0000005,True,Abetalipoproteinemia,"[Bassen Kornzweig syndrome,Microsomal triglyce..."
4,GARD:0000006,True,Acromesomelic dysplasia,[Acromesomelic dwarfism]


- make sure is_rare column is a true boolean data type.  It is

In [6]:
neo4j_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6305 entries, 0 to 6304
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   d.gard_id   6305 non-null   object
 1   d.is_rare   6305 non-null   bool  
 2   d.name      6305 non-null   object
 3   d.synonyms  5033 non-null   object
dtypes: bool(1), object(3)
memory usage: 154.1+ KB


- filter out the diseases in the GARD database that are specifically noted as not being rare

In [7]:
neo4j_df = neo4j_df[neo4j_df['d.is_rare']==True]

- Using each disease name, search every subreddit's description text for matches with that disease name.  Save each index (paired with disease name matched in the search) of every subreddit that matches with a disease name so we can grab it later

- used the parameter case=False in the contains() method so the search was not case sensitive.  The search takes longer because of this but our data has upper and lowercases so this was necessary 

In [8]:
# match_dict = {}
# for name in list(neo4j_df['d.name']):
#     boolean_array = subreddit_df['description'].str.contains(name, na=False, regex=False, case=False)
#     match_idx = subreddit_df.index[boolean_array]
#     if len(list(match_idx))>0:
#         match_dict[name] = list(match_idx)
        
# a_file = open("..Data/GARD_and_subreddit_matches_dict_data.json", "w")
# json.dump(match_dict, a_file)
# a_file.close()

- Load json file instead of running the search again for successive work

In [7]:
with open('../Data/GARD_and_subreddit_matches_data_interim.json') as f:
    match_dict = json.load(f)

- Printed out every disease name that had matches and the matches (subreddits) themselves.  I manually went over this output to filter out subreddits that matched but that were not relevant to rare disease

- Make tuples so that each index match has the subreddit name and disease search term together with it.  This will allow us to create our dataframe later

In [10]:
list_of_tuples = []
for key,value in match_dict.items():

    print('term '+key+' has '+str(len(value))+' matches')
    print(subreddit_df.iloc[value]['title'].values)
    print(subreddit_df.iloc[value]['description'].values)
    print(subreddit_df.iloc[value]['name'].values)
    print('\n')
    
    for idx in value:
        list_of_tuples.append((idx,key,subreddit_df.iloc[idx]['name']))

term Agnosia has 3 matches
['community for faceblind people!' 'The face blindness subreddit'
 'For the non visualisers']
['Prosopagnosia, or face blindness, affects an estimated 2.5% to 10% of the population to some degree.\n\nFace blindness is a disorder of face perception where the ability to recognize faces is impaired, while the ability to recognize other objects may be relatively intact.\n\nFace blindness may be caused by brain injury or may be inherited. Many faceblind people develop elaborate mechanisms for navigating social situations, and sometimes develop social anxiety problems because of it.\n\nThis community is a place to help each other and raise awareness of prosopagnosia.'
 "[Prosopagnosia,](https://www.faceblind.org/research/) or face blindness, is an neurological disorder characterized by the inability to recognize or differentiate faces. Estimates suggest [as many as 1 in 50 people](https://onlinelibrary.wiley.com/doi/abs/10.1348/174866407X231001) are born with the c

["A place for all things BSD.\n\nGet it!\n\n* [DragonFlyBSD](http://www.dragonflybsd.org/download/)\n* [FreeBSD](http://www.freebsd.org/where.html)\n* [NetBSD](http://www.netbsd.org/releases/)\n* [OpenBSD](http://www.openbsd.org/ftp.html)\n* [TrueOS (formerly PC-BSD)](https://www.trueos.org/downloads/)\n\n***\n\nSimilar reddits\n\n* /r/unix\n* /r/dragonflybsd\n* /r/freebsd\n* /r/netbsd\n* /r/openbsd\n* /r/trueos (formerly /r/pcbsd)\n***\n\nMan(ual) Pages\n\n* [DragonFlyBSD](http://leaf.dragonflybsd.org/cgi/web-man)\n* [FreeBSD](http://www.freebsd.org/cgi/man.cgi)\n* [NetBSD](http://netbsd.gw.com/cgi-bin/man-cgi/man?+ANY+NetBSD-current)\n* [OpenBSD](http://www.openbsd.org/cgi-bin/man.cgi)\n\n***\n\n[Tutorial on reddit's markup rules](http://www.reddit.com/r/raerth/comments/cw70q/reddit_comment_formatting)."
 '#Welcome to the hub of Food on reddit!\n\nThanks for making us over ~~16~~ ~~17~~ **18 MILLION STRONG!**\n\n---\n\n#####Upcoming AMAs\n\n* We currently have no scheduled AMAs\n    

['BSD' 'food' 'recipes' 'canada' 'photoshop' 'death' 'disability' 'deaf'
 'hockey' 'fastfood' 'stocks' 'lua' 'Sparta' 'Anxiety' 'LiverpoolFC'
 'OttawaSenators' 'MMJ' 'Blind' 'ForgottenBookmarks' 'nhl' 'chubby'
 'prolife' 'Needafriend' 'asl' 'asexuality' 'Nissan' 'electricvehicles'
 'postpunk' 'CannabisExtracts' 'lostgeneration' 'penguins'
 'metacirclejerk' 'leafs' 'highschool' 'AfterEffects' 'SanJoseSharks'
 'DetroitRedWings' 'lotr' 'losangeleskings' 'amiugly' 'Handjob' 'ps3bc2'
 'toosoon' 'PropagandaPosters' 'GetMotivated' 'BostonBruins' 'rangers'
 'devils' 'Predators' 'ColoradoAvalanche' 'FloridaPanthers'
 'TampaBayLightning' 'naturopathy' 'realmadrid' 'Warhammer40k' 'chelseafc'
 'BlackMetal' 'realtrees' 'coys' 'SimCity' 'rya11111' 'GriefSupport'
 'ForeverAlone' 'Spartanburg' 'inmemoryof' 'ocala' 'avfc' 'frederickmd'
 'torontoraptors' 'Clarksville' 'Pescetarian' 'NUFC' 'Trivium' 'oceans'
 'woodman' 'AubreyMaturinSeries' 'edmproduction' 'audiology'
 'Torontobluejays' 'hearing' 'iosgam

['Prader-Willi Syndrome']
["Prader-Willi Syndrome (PWS) is a rare genetic disorder, a damaged, missing or incomplete chromosome 15, that is responsible for the development of the Hypothalamus. \n\nPeople with PWS have an insatiable hunger, poor muscle tone, poor body thermostat, high pain threshold and more. PWS is commonly associated in the media as 'those that cannot stop eating'"]
['praderwilli']


term Rickets has 15 matches
['Cricket' 'Lemon : All the Way' 'Eating Insects'
 'Pet food for all the animals!' 'Edible Insects' '/r/Instantregret'
 'A game in a glimpse.' 'Cricket Shitpost' "Reddit's first!"
 'Reverend Insanity' 'musical_meditation'
 'The policy equivalent of r/HiddenChangelog' 'VitaminD3'
 'Place for discussion of insects (and other creepy crawlies) within New Jersey '
 'fuckcrickets']
['&gt;**Links**\n\n&gt;* **Related Subreddits**\n* [CricketGifs] (https://www.reddit.com/r/cricketgifs)\n* [FrontFootPorn](https://www.reddit.com/r/frontfootporn)\n* [DeathRattlePorn](http

['polio']


term Polycythemia vera has 1 matches
['Myeloproliferative Neoplasms']
['This subreddit was created both to store and to share information about Myeloproliferative Neoplasms (Myeloproliferative Disorders). These include Essential Thrombocythemia, Polycythemia Vera, Myelofibrosis, and Chronic Myelogenous Leukemia, and others.\n\nMyeloproliferative Neoplasms are progressive blood cancers with a prevalence of roughly 150,000 to 200,000 people in the United States.']
['MPN']


term Progeria has 1 matches
['Progerion']
['Welcome to Progerion, the land of the Progerian Empire!']
['Progerion']


term Proteus syndrome has 1 matches
['Solve the NF Puzzle']
['Facts &amp; Statistics\nÂ· NF has been classified into three distinct types; NF1, NF2 and schwannomatosis. They are caused by different genes, located on different chromosomes.\n\nÂ· NF1 is the most common neurological disorder caused by a single gene; occuring in one in every 3,000 children born.\n\nÂ· NF2 is a rarer type, occur

["r/cholesteatoma is a place for community help and general discussion about tinnitus. Share your experiences with others who can relate. A cholesteatoma is an abnormal, noncancerous skin growth that can develop in the middle section of your ear, behind the eardrum. It may be a birth defect, but it's most commonly caused by repeated middle ear infections. A cholesteatoma often develops as a cyst, or sac, that sheds layers of old skin."]
['cholesteatoma']


term Morphea has 1 matches
['scleroderma ']
['**Scleroderma** is an autoimmune disease which attacks connective tissue, causing an over-production of collagen and leaving behind hard scar tissue.  \n\nScleroderma has two major classifications:  *Localized Scleroderma* and *Systemic Scleroderma*.  Subtypes of these classifications, each having its own characteristics and prognosis, are based upon the groupings of symptoms.  Within Localized Scleroderma, two forms exist:  *Morphea* and *Linear Scleroderma*.  Within Systemic Scleroderma

- a few of these rare disease names were too common and had tons of matches that were hard to go through manually. So I just removed all tuples that include these search terms

In [13]:
list_of_tuples = [(x,y,z) for x,y,z in list_of_tuples if y not in ['EAF', 'N syndrome', 'Noma', 'C syndrome']]

- from the remaining possible subreddits, I manually created list of all subreddit matches that I deemed as appropriate rare disease subreddits

In [11]:
match_list = "faceblind,Prosopagnosia,PeriodicParalysis,AcousticNeuroma,Microtia,BrittleBones,collagendisorders,CADASIL,Autoinflammatory,NCAH,ShortBowelSyndrome,Staphacne,poliosis,piebaldism_inhumans,iih,CRPS,neurofibromatosis,HNPPsupport,praderwilli,AddisonsDisease,Albinism,A1AD,ALS,AngelmanSyndrome,mds,aplasticanemia,Apraxia,primaryimmune,BellsPalsy,Angioedema,CMT,SynovialChondro,guillainbarre,cll,CLL_Leukemia,CML,CVID,ConversionDisorder,PTEN,CysticFibrosis,Cystic_Fibrosis,cystichygroma,Dengue_Fever,dermatomyositis,NMOdisease,EbsteinsAnomaly,ectopicpregnancy,EctopicSupportGroup,selectivemutism,E_L,fabrysdisease,TMAU,glossopharyngeal,G6PD,Hemophilia,MPN,HHT,Keratoconus,DupuytrenDisease,CJD,marfans,MCAS,mastcelldisease,MastCellDiseases,scleroderma,Sanfilippo,Narcolepsy,Osteosarcoma,PolandSyndrome,Pyodermagangrenosum,ThrivingandShaking,spinabifida,Hydrocephalus,spinalmuscularatrophy,stomachcancer,Smasyndrome,thalassemia,TrigeminalNeuralgia,MuscularDystrophy,FSHD,Strabismus,ADPKD,Angioedema,idiopathichypersomnia,lqts,pancreaticcancer,hyperacusis,PigeonChest,openpiriformis,plagiocephaly,CDH1,cholesteatoma,ehlersdanlos,SticklerSyndrome,misophoniasupport,misophonia,VisualSnowSyndrome,Gastroparesis,MonoHearing".split(',')
match_list = list(set(match_list))

- filter through our tuples to remove any tuples with a subreddit name not in the approved list of matches I just created above

In [12]:
tuples_that_match = [(x,y,z) for x,y,z in list_of_tuples if z in match_list]

- let's check out the remaining tuples.  They should be only rare disease related.  Looks pretty good

In [14]:
tuples_that_match

[(21508, 'Agnosia', 'faceblind'),
 (54789, 'Agnosia', 'Prosopagnosia'),
 (118475, 'Hyperkalemic periodic paralysis', 'PeriodicParalysis'),
 (123820, 'Acoustic neuroma', 'AcousticNeuroma'),
 (48384, 'Microtia-Anotia', 'Microtia'),
 (93989, 'Osteogenesis imperfecta', 'BrittleBones'),
 (241355, 'Osteogenesis imperfecta', 'collagendisorders'),
 (44233, 'CADASIL', 'CADASIL'),
 (101234,
  'Neonatal Onset Multisystem Inflammatory disease',
  'Autoinflammatory'),
 (194690, 'Congenital adrenal hyperplasia', 'NCAH'),
 (165178, 'Short bowel syndrome', 'ShortBowelSyndrome'),
 (57987, 'Glioblastoma', 'Staphacne'),
 (104333, 'Piebaldism', 'poliosis'),
 (287768, 'Piebaldism', 'piebaldism_inhumans'),
 (36911, 'Idiopathic intracranial hypertension', 'iih'),
 (14447, 'Complex regional pain syndrome', 'CRPS'),
 (18971, 'Schwannoma', 'neurofibromatosis'),
 (18971, 'Schwannomatosis', 'neurofibromatosis'),
 (130024,
  'Hereditary neuropathy with liability to pressure palsies',
  'HNPPsupport'),
 (104333, 'W

- Create a new dataframe of only the chosen rare disease subreddits and the information associated with each subreddit we took from the subreddit master archive

In [15]:
GARD_matches_df = subreddit_df.iloc[list(set([x for x,y,z in tuples_that_match]))].copy()

- As of now, the only columns for our new dataframe are the ones from the original subreddit dataframe.  Let's create a new column that will also add the rare disease search term (from the database) that matched that subreddit's description. It is important we can tie each subreddit to the disease in the database it was chosen from

- Initialize the column to an empty string then all rows should be updated to the correct value next

In [16]:
GARD_matches_df['search_term'] = ''
GARD_matches_df['GARD_id'] = ''

- Update each row using our tuples

- Since any given subreddit entry could have matched to multiple disease terms, this new column 'search_term' can have multiple values

In [17]:
for x,y,z in tuples_that_match:
    GARD_matches_df.loc[GARD_matches_df['name']==z, 'search_term'] += y + ','
    GARD_matches_df.loc[GARD_matches_df['name']==z, 'GARD_id'] += neo4j_df.loc[neo4j_df['d.name']==y]['d.gard_id'].values[0] + ','

- In the end, we have 98 subreddits that qualify as true rare disease subreddits

In [18]:
GARD_matches_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 5120 to 76275
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        98 non-null     object 
 1   description  98 non-null     object 
 2   subscribers  98 non-null     int64  
 3   created_utc  98 non-null     float64
 4   name         98 non-null     object 
 5   search_term  98 non-null     object 
 6   GARD_id      98 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 6.1+ KB


- reorder and rename our columns for a better display

In [19]:
GARD_matches_df = GARD_matches_df[['name','search_term', 'description', 'title', 'subscribers', 'created_utc', 'GARD_id']]
GARD_matches_df.columns = ['subreddit','GARD_diseas(es)','description', 'display_title', 'subscribers', 'created_utc', 'GARD_id(s)']

- looks good

In [20]:
GARD_matches_df.head()

,r/,GARD_diseas(es),description,display_title,subscribers,created_utc,GARD_id(s)
5120,Keratoconus,"Keratoconus,Pellucid marginal degeneration,",## What is Keratoconus?\n\nKeratoconus (KC) is...,Keratoconus | Queratocono | Keratocone | Chera...,6493,1.252794e+09,"GARD:0006824,GARD:0011895,"
123904,DupuytrenDisease,"Ledderhose disease,",Community for the discussion of Dupuytren's co...,DupuytrenDisease,76,1.551793e+09,"GARD:0006873,"
21508,faceblind,"Agnosia,","Prosopagnosia, or face blindness, affects an e...",community for faceblind people!,78,1.328415e+09,"GARD:0000008,"
54789,Prosopagnosia,"Agnosia,","[Prosopagnosia,](https://www.faceblind.org/res...",The face blindness subreddit,1580,1.403747e+09,"GARD:0000008,"
92165,PolandSyndrome,"Poland syndrome,",A place for folks with Poland Syndrome to meet...,"Poland Syndrome: A congenital, non hereditary ...",144,1.495482e+09,"GARD:0007412,"


- save our new dataframe so it can be used moving forward in other notebooks

In [22]:
# GARD_matches_df.to_csv('Data/GARD_subreddit_matches_df.csv', index=False)